In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Bloom176-petals installation (model running not working)

In [2]:
#!pip install git+https://github.com/bigscience-workshop/petals

In [3]:
# import torch
# from transformers import BloomTokenizerFast 
# from petals import DistributedBloomForCausalLM

# Bloom176-petals model creation

In [4]:
# MODEL_NAME = "bigscience/bloom-petals"
# tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME)
# model = DistributedBloomForCausalLM.from_pretrained(MODEL_NAME)

In [5]:
# inputs = tokenizer("", return_tensors="pt")["input_ids"].cuda()
# outputs = model.generate(inputs, max_new_tokens=5)

In [6]:
# print(tokenizer.decode(outputs[0]))

# Dataset installation

I uploaded this dataset to kaggle, so we can use it immediatelly. We will use pandas to convert jsonl files to the dataset, since the dataset is rather small.

In [7]:
import pandas as pd

#train = pd.read_json(path_or_buf="/kaggle/input/gsm8k-dataset/train.jsonl", lines=True)
test = pd.read_json(path_or_buf="/kaggle/input/gsm8k-dataset/test.jsonl", lines=True)

As we can see, there are 1319 questions in the test without any additional annotations. And there are also answers, where sometimes there are calculator annotations starting and ending with << >>. The numeric answer is in the end after ####.

In [8]:
test

,question,answer
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t..."
...,...,...
1314,John had a son James when he was 19. James is...,Dora is 12-3=<<12-3=9>>9\nSo James is 9*2=<<9*...
1315,There are some oranges in a basket. Ana spends...,There are 60 minutes in an hour. Ana peels an ...
1316,Mark's car breaks down and he needs to get a n...,The discount on the radiator was 400*.8=$<<400...
1317,"Farmer Brown has 20 animals on his farm, all e...",Let C be the number of chickens.\nThere are 20...


Simple preprocessing to extract all the answers from the test

In [9]:
def get_num_answer_test(text):
    return text.split()[-1]

test["numeric_answer"] = test.apply(lambda x: get_num_answer_test(x["answer"]), axis=1)

In [10]:
test

,question,answer,numeric_answer
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...,70000
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t...",20
...,...,...,...
1314,John had a son James when he was 19. James is...,Dora is 12-3=<<12-3=9>>9\nSo James is 9*2=<<9*...,8
1315,There are some oranges in a basket. Ana spends...,There are 60 minutes in an hour. Ana peels an ...,5
1316,Mark's car breaks down and he needs to get a n...,The discount on the radiator was 400*.8=$<<400...,230
1317,"Farmer Brown has 20 animals on his farm, all e...",Let C be the number of chickens.\nThere are 20...,5


# Creating hypothesis

There are many options how to test CoT samples. We can change the number of examplars, the order of them and just the examplars. All these params can impact on the performance. It's hard to do all the tests due to computational limitations. It's known, that the number of examplars do not significantly increases the score. However, annotators quality can be important. For simplification let's fix the order and the number of examplars. For example, we take 4 of them. However, I suggest to create 2 versions (to imitate annotators.

Required methods that we are going to test:

1. CoT (4 examplars, fixed order, 2 versions)
2. CoT ensemble (4 examplars, same order as in the 1 item, 2 versions): do top-k (40) sampling to get several answers, then do the majority vote.

In the 2 item I also suggest to take recommended params from the paper, since there are too many options to try otherwise, moreoever, authors wrote that models performance is quite robust to hyperparams changes.

Bonus methods that are interesting (for me) to try:

3. CoT: 1 question, several correct reasoning paths with correct answers. Then do majority vote on them. Prompt example:

```
Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder for $2 per egg. How much does she make every day? 
A: She has 16 - 3 - 4 = 9 eggs left. So she makes $2 * 9 = $18 per day. The answer is $18.
A: She eats 3 for breakfast, so she has 16 - 3 = 13 left. Then she bakes muffins, so she has 13 - 4 = 9 eggs left. So she has 9 eggs * $2 = $18. The answer is $18.
A: This means she uses 3 + 4 = 7 eggs every day. So in total she sells 16 - 7 = 9 eggs. She sells the remainder for $2 per egg, so she makes 9 * $2 = $18 per day. The answer is $18.


Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
```

4. CoT with directing phrases. Prompt example:

```
Use mathemathical operations and write step by step the explanation of the following math problem solution:
Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder for $2 per egg. How much does she make every day?

Here is the solution explanation:
A: She has 16 - 3 - 4 = 9 eggs left. So she makes $2 * 9 = $18 per day. The answer is $18.

Use mathemathical operations and write step by step the explanation of the following math problem solution:
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?

Here is the solution explanation:
```


# Creating prompts for the dataset

In [11]:
def get_prompt_cot_anot1(text):
    '''
    Method for the first hypothesis and the first set of tasks (annotator)
    '''
    
    # Fixing prompt examplars
    temp = ("Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\n"
            "A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.\n"
            "Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\n"
            "A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.\n"
            "Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\n"
            "A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.\n"
            "Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?\n"
            "A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.\n"
            "Q: "
           )
    return temp + text

In [12]:
def get_prompt_cot_anot2(text):
    '''
    Method for the first hypothesis and the second set of tasks (annotator)
    '''
    
    # Fixing prompt examplars
    temp = ("Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\n"
            "A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so in total he has 7 + 2 = 9 toys. The answer is 9.\n"
            "Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?\n"
            "A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 = 20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers. The answer is 29.\n"
            "Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\n"
            "A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.\n"
            "Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\n"
            "A: She bought 5 bagels for $3 each. This means she spent 5 * $3 = $15 on the bagels. She had $23 in beginning, so now she has $23 - $15 = $8. The answer is 8.\n"
            "Q: "
           )
    return temp + text

In [13]:
def get_prompt_cot_many_paths(text):
    '''
    Method for the 3rd hypothesis with several reasonings
    '''
    
    temp = ("Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?\n"
            "A: In the beginning, Betty has only $100 / 2 = $50. Betty's grandparents gave her $15 * 2 = $30. This means, Betty needs 100 - 50 - 30 - 15 = $5 more. The answer is 5.\n"
            "A: She had only half of the money she needs, that is, she needs $100 / 2 = $50. Her parents gave her $15. Grandparents gave her $15 * 2 = $30. This means, Betty needs 100 - 50 - 15 - 30 = $5 more. The answer is 5.\n"
            "A: Betty had only half of the wallet cost, she had $100 / 2 = $50. Then grandparents gave her twice much as parents did, $15 * 2 = $30. Betty has 50 + 30 + 15 = $95. Wallet costs 100. She needs 100 - 95 = $5 more. The answer is 5.\n"
            "\n"
            "Q: "
           )
    return temp + text

In [14]:
def get_prompt_cot_directions(text):
    '''
    Method for the 4th hypothesis with directions
    '''
    
    temp = ("Use mathemathical operations and write step by step the explanation of the following math problem solution:\n"
            "Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\n"
            "A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.\n"
            "Use mathemathical operations and write step by step the explanation of the following math problem solution:\n"
            "Q: "
           )
    return temp + text + "\nHere is the solution explanation:"

In [15]:
test["CoT_anot1"] = test.apply(lambda x: get_prompt_cot_anot1(x["question"]), axis=1)

In [16]:
test["CoT_anot2"] = test.apply(lambda x: get_prompt_cot_anot2(x["question"]), axis=1)

In [17]:
test["CoT_paths"] = test.apply(lambda x: get_prompt_cot_many_paths(x["question"]), axis=1)

In [18]:
test["CoT_directions"] = test.apply(lambda x: get_prompt_cot_directions(x["question"]), axis=1)

In [19]:
test.to_csv("test_prompts.csv", index=False)